# Scraping af FT

In [3]:
from bs4 import BeautifulSoup as bs
import re 
import os
import lxml
import xml.etree.ElementTree as ET
from lxml import objectify
from datetime import datetime
import numpy as np 
import pandas as pd 
import requests
import ftplib
from io import BytesIO
import tqdm

##### Tomme lister

In [4]:
dirs = []
sub_dir = []
files = []
file_name = []
paths = []
raw_xml_list = []

suppe = []

### Login til ftp serveren og henter en liste over biblioteker

In [5]:
ftp_url = "oda.ft.dk"
ftp_dir = "/ODAXML/Referat/samling/"

In [6]:
ftp = ftplib.FTP(ftp_url)
ftp.login("anonymous", "wpg345@alumni.ku.dk")
ftp.set_pasv(True)
ftp.cwd(ftp_dir)
ftp.dir(dirs.append)

In [7]:
for i in range (0, len(dirs), 1):    # bygger en liste med underbiblioteker
    sub_dir.append(ftp_dir+dirs[i][-5:]+'/') 
    
for i in range (0, len(sub_dir), 1): # bygger en liste med filnavne
    ftp.cwd(sub_dir[i])
    ftp.dir(files.append)

for i in range(0, len(files)):       # extract filnavn
    file_name.append(files[i][39:])   

for i in range(0, len(file_name)):   # opbygning af den fulde sti og filnavn
    paths.append(ftp_dir+file_name[i][0:5]+'/'+file_name[i])

### Overførsel af de enkelte filer fra ftp://oda.ft.dk, og raw_xml skrives til liste

In [8]:
for i in tqdm.tqdm(range(0, len(paths))):
    r = BytesIO()
    ftp.retrbinary(f"RETR {paths[i]}", r.write)
    raw_xml_list.append(r.getvalue())
    r.close()

100%|██████████| 1489/1489 [02:51<00:00,  8.68it/s]


# Define path function

In [50]:
# extract_path VIRKER
repeats = []

def extract_path(xml_list):
    
    for xml in xml_list: 
        soup = bs(xml, 'lxml')
        repeats.append(len(soup.find_all("dagsordenpunkt")))
    
    for i in range(0, len(xml_list)): 
        p = paths[i]
        path.extend([p] * repeats[i]) 
        #extend acts as append but multiplied by n times
    
    return None

Bemærk vi er interesseret i len(xml_list) modsat de næste funktioner, hvor det ikke behøver spille nogen rolle. Derfor vælger vi ikke at integrere extract_path ind i extract_metadata. 

# Define metadata function

In [51]:
# vers1 virker
def extract_metadata(xml):
    soup = bs(xml, 'lxml')
    
    d = soup.find("dateofsitting").text[:10]
    m = soup.find("meetingnumber").text
    s = soup.find("parliamentarysession").text
    g = soup.find("parliamentarygroup").text
    l = soup.find("location").text
    
    # antal gange de samme metadata gentages for hvert dagordenpunkt
    repeats = len(soup.find_all("dagsordenpunkt")) 
    
    for repeat in range(0, repeats):
        date.append(d)
        meetingnumber.append(m)
        session.append(s)
        group.append(g)
        location.append(l)
    
    return None

# Define content function 

In [52]:
def extract_content(xml): 
    soup = bs(xml, 'lxml')
    dagsordenpunkter = soup.find_all("dagsordenpunkt") # liste med dagsordenpunkter
    #print(f"Antal dagsordenpunkter: {len(dagsordenpunkter)}")
    
    for dp in dagsordenpunkter: # find raw xml-sætninger for hvert dagsordenpunkt
        sentences_raw = dp.find_all("linea") # liste med raw sætninger 
        #print(f"Antal sentences_raw: {len(sentences_raw)}")
        
        sentences_text = []
        
        for sr in sentences_raw: # for hver raw sætning, kør .text af sætningen
            sentences_text.append(sr.get_text(separator = ' ')) # gem clean sætninger til liste
        
        content.append(' '.join(sentences_text).strip()) # join, strip og append clean sætninger til liste
        #print(f"Antal content: {len(content)}")

    return None

# Sample run

In [13]:
xml_sample = raw_xml_list[0:1]

In [ ]:
for xml in xml_sample: 
    extract_metadata(xml) # meta

In [38]:
for xml in tqdm.tqdm(xml_sample): 
    extract_content(xml) # content

100%|██████████| 1/1 [00:02<00:00,  2.00s/it]


'Meddelelser fra formanden Tredje næstformand  (Holger K. Nielsen) Mødet er åbnet. Finansudvalget har afgivet: Betænkning og indstilling vedrørende forslag til folketingsbeslutning i henhold til grundlovens § 47 med hensyn til statsregnskabet for finansåret 2008. (Beslutningsforslag nr. B 248). Betænkningen og indstillingen vil fremgå af www.folketingstidende.dk.'

# Full run

In [53]:
# empty lists (columns)
date = []
content = []
meetingnumber = []
session = []
group = []
location = []
path = []

In [54]:
for xml in tqdm.tqdm(raw_xml_list): 
    extract_metadata(xml)

100%|██████████| 1489/1489 [10:18<00:00,  2.41it/s]


In [55]:
for xml in tqdm.tqdm(raw_xml_list): 
    extract_content(xml)

100%|██████████| 1489/1489 [10:56<00:00,  2.27it/s]


In [56]:
extract_path(tqdm.tqdm(raw_xml_list))

100%|██████████| 1489/1489 [09:59<00:00,  2.48it/s]


### title kladdder

In [ ]:
soup = bs(sample1, 'lxml')
test = soup.find("punkttekst").find_all("char")
test

In [ ]:
sample1 = raw_xml_list[0]

In [ ]:
sample1 = raw_xml_list[0]
soup = bs(sample1, 'lxml')

# Make DataFrame

In [67]:
df = pd.DataFrame([session, meetingnumber, date, location, group, content, path]).transpose()
df.columns = ['session', 'meetingnumber', 'date', 'location', 'group', 'content', 'path']

In [68]:
df

,session,meetingnumber,date,location,group,content,path
0,20091,100,2010-05-31,Folketingssalen,Folketinget,Meddelelser fra formanden Tredje næstformand ...,/ODAXML/Referat/samling/20091/20091_M100_helem...
1,20091,100,2010-05-31,Folketingssalen,Folketinget,Det første punkt på dagsordenen er: 1) 1. beha...,/ODAXML/Referat/samling/20091/20091_M100_helem...
2,20091,100,2010-05-31,Folketingssalen,Folketinget,Det næste punkt på dagsordenen er: 2) 1. behan...,/ODAXML/Referat/samling/20091/20091_M100_helem...
3,20091,100,2010-05-31,Folketingssalen,Folketinget,Det næste punkt på dagsordenen er: 3) 1. behan...,/ODAXML/Referat/samling/20091/20091_M100_helem...
4,20091,100,2010-05-31,Folketingssalen,Folketinget,Det næste punkt på dagsordenen er: 4) 1. behan...,/ODAXML/Referat/samling/20091/20091_M100_helem...
...,...,...,...,...,...,...,...
16684,20211,9,2021-10-26,Folketingssalen,Folketinget,Det næste punkt på dagsordenen er: 4) 1. behan...,/ODAXML/Referat/samling/20211/20211_M9_helemoe...
16685,20211,9,2021-10-26,Folketingssalen,Folketinget,Det næste punkt på dagsordenen er: 5) 1. behan...,/ODAXML/Referat/samling/20211/20211_M9_helemoe...
16686,20211,9,2021-10-26,Folketingssalen,Folketinget,Det næste punkt på dagsordenen er: 6) 1. behan...,/ODAXML/Referat/samling/20211/20211_M9_helemoe...
16687,20211,9,2021-10-26,Folketingssalen,Folketinget,Det sidste punkt på dagsordenen er: 7) 1. beha...,/ODAXML/Referat/samling/20211/20211_M9_helemoe...


# Examining the empty content rows 

# Save df to csv

In [69]:
df.to_csv('FT.csv', index = False)

In [70]:
df = pd.read_csv('FT.csv')

In [81]:
stopwords = ['er', 'og', 'der', 'på']

In [86]:
len(df['content'])

16689

In [96]:
sample = df[0:3]

In [95]:
sample

,session,meetingnumber,date,location,group,content,path
0,20091,100,2010-05-31,Folketingssalen,Folketinget,Meddelelser fra formanden Tredje næstformand ...,/ODAXML/Referat/samling/20091/20091_M100_helem...
1,20091,100,2010-05-31,Folketingssalen,Folketinget,Det første punkt på dagsordenen er: 1) 1. beha...,/ODAXML/Referat/samling/20091/20091_M100_helem...
2,20091,100,2010-05-31,Folketingssalen,Folketinget,Det næste punkt på dagsordenen er: 2) 1. behan...,/ODAXML/Referat/samling/20091/20091_M100_helem...


In [125]:
## converts lists of list to ONE list: import itertools; list(itertools.chain(*mylist))